# Оценка эмоциональной окраски сообщений

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

## Загрузка данных

In [2]:
cols=["id","date","mane","text","type","trep","trtw","tfav","stcount","fol","frien","listcount"]

In [3]:
pos = pd.read_csv("positive.csv",sep=";", names=cols)
pos["label"] = 1
neg = pd.read_csv("negative.csv",sep=";", names=cols)
neg["label"] = 0

In [4]:
data = pd.concat([pos, neg], ignore_index=True)

In [5]:
# 5 примеров строк
data.head()

,id,date,mane,text,type,trep,trtw,tfav,stcount,fol,frien,listcount,label
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0,1
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2,1
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0,1
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0,1
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1,1


In [6]:
# пример отзыва
print(data.text[111])

@__devyatyi__ да вот и получается, что хоть и москвичи,а всё как по понятиям привыкли,так только и могут.Позорище.Ещё и начальство вызывают)


## Размеры классов

In [3]:
#data.groupby("label").count()

## Наиболее часто встречающиеся слова

In [7]:
# для позитивных отзывов
positive_reviews = ' '.join(data.text[data.label == 1])
positive_counter = Counter(positive_reviews.split())

for word in positive_counter.most_common(10):
    print(word[0], word[1])

и 28300
в 26652
не 26545
я 22498
RT 21635
на 18239
с 14131
что 12776
:) 11674
:D 9977


In [8]:
# для негативных
negative_reviews = ' '.join(data.text[data.label == 0])
negative_counter = Counter(negative_reviews.split())

for word in negative_counter.most_common(10):
    print(word[0], word[1])

не 40981
я 26056
в 25472
и 24836
:( 19351
на 17016
RT 15573
что 13852
а 12464
меня 10964


## Обучаем первую модель

In [9]:
# преобразуем текст в числа
tfidf = TfidfVectorizer()
f = tfidf.fit_transform(data["text"])
print(f.shape)

(226834, 294600)


In [10]:
#pd.DataFrame(f[:5, :].todense(), columns=tfidf.get_feature_names()).head()

In [11]:
%%time
# делим на обучение и тест
trX, teX, trY, teY, trDF, teDF = train_test_split(
    f, data["label"], data, test_size=0.2, stratify=data["label"], random_state=42
)
clf = LogisticRegression(n_jobs=1, solver='liblinear')
clf.fit(trX, trY)

CPU times: user 2.68 s, sys: 28 ms, total: 2.71 s
Wall time: 2.78 s


In [12]:
# добавляем колонку с вероятностью положительного отзыва
teDF = teDF.copy()
teDF["predicted_proba"] = list(map(str, clf.predict_proba(tfidf.transform(teDF["text"]))[:, 1]))

In [13]:
teDF.tail(10)

,id,date,mane,text,type,trep,trtw,tfav,stcount,fol,frien,listcount,label,predicted_proba
142482,412566596276592640,1387198516,__________C__,RT @Fuma_like: хх\n\n.\n\n.\n\n ...,-1,0,1,0,297,33,33,0,0,0.5779432087773065
30081,409634283792629760,1386499398,NadezhdaVinokur,RT @vitabakan: @abakanskiy рачком не стань слу...,1,0,1,0,16448,427,366,5,1,0.6798347196840785
127056,410751105740849153,1386765669,noizer__,"RT @kalininaaaaaaa: @noizer__ да,было бы очень...",-1,0,1,0,19290,681,310,21,0,0.6197052784281845
77695,410712435348492288,1386756449,cristina_bardak,Смотрим Марли и Я. Соседи уже скоро ментов выз...,1,0,0,0,311,9,14,0,1,0.9201304870668058
24366,409512898713899008,1386470458,luqygixekof,Дети — радость жизни... Но порой они так доста...,1,0,0,0,96,83,80,0,1,0.514033565407247
207452,422092311968112640,1389469623,MaxSneg1,RT @Damiriki: Блин что то я взгрустнул(( без л...,-1,0,1,0,488,25,23,0,0,0.46833124272548654
144247,412741527476387840,1387240223,bicijupoxoje,RT @pehenytizuwo: Сегодня выходной?! Хрена-с-д...,-1,0,4,0,1348,222,223,0,0,0.17620649156364177
97788,411045609358262272,1386835884,Kaia_Jenny,@Devushka_Radio @loontik_loo а чё? Я частенько...,1,0,0,0,7422,292,69,8,1,0.720376440213133
109250,411171848345640960,1386865982,AntonShahmatenk,RT @financerf: Российский #рубль наконец получ...,1,0,341,0,17,7,61,0,1,0.954232841727717
183059,417353175792762880,1388339725,subxankulova199,"Заебал честное слово всё запрещать,надоело( Св...",-1,0,0,0,1523,21,20,1,0,0.31785316906405525


## Потестируем модель

In [14]:
clf.predict_proba(tfidf.transform([u"всё отлично было сегодня"]))[:,1]

array([0.83161185])

In [15]:
clf.predict_proba(tfidf.transform([u"сильно болела голова"]))[:,1]

array([0.04093035])

In [16]:
clf.predict_proba(tfidf.transform([u"ужасно болела голова"]))[:,1]

array([0.00703031])

In [17]:
clf.predict_proba(tfidf.transform([u"смотрел интересный фильм"]))[:,1]

array([0.89104143])

## Формальные метрики качества

In [18]:
print("train accuracy:", accuracy_score(trY, clf.predict(trX)))
print("test accuracy:", accuracy_score(teY, clf.predict(teX)))

train accuracy: 0.8467214424661234
test accuracy: 0.7608614190931734


## Посмотрим на веса модели

In [19]:
coefs = sorted(zip(list(np.array(tfidf.get_feature_names())[clf.coef_[0] != 0]), clf.coef_[0][clf.coef_[0] != 0]),
               key=lambda x: -x[1])
coefs = pd.DataFrame(coefs, columns=["ngram", "weight"])

In [20]:
coefs.head(30)

,ngram,weight
0,dd,10.794644
1,ddd,8.199638
2,ахаха,5.858155
3,ахах,5.727581
4,спасибо,5.319622
5,ахахах,5.220887
6,dddd,5.075977
7,приятно,4.960392
8,xd,4.954467
9,обожаю,4.316036


In [21]:
coefs.tail(30)[::-1]

,ngram,weight
254625,o_o,-10.347952
254624,о_о,-10.310261
254623,обидно,-7.534158
254622,cio_optimal,-7.514311
254621,жаль,-6.953999
254620,to_over_kill,-6.906745
254619,скучаю,-6.875654
254618,печально,-6.556959
254617,99,-6.262098
254616,грустно,-6.249307


## Попробуем фильтровать стоп-слова

In [22]:
# посчитаем встречаемость всех слов
all_words = ' '.join(data.text).lower()
word_counts = Counter(all_words.split())

In [23]:
word_counts

Counter({'@first_timee': 1,
         'хоть': 1934,
         'я': 59961,
         'и': 58158,
         'школота,': 2,
         'но': 12701,
         'поверь,': 17,
         'у': 21573,
         'нас': 3875,
         'то': 9637,
         'же': 8816,
         'самое': 935,
         ':d': 9991,
         'общество': 40,
         'профилирующий': 1,
         'предмет': 34,
         'типа)': 3,
         'да,': 1494,
         'все-таки': 306,
         'он': 6282,
         'немного': 774,
         'похож': 121,
         'на': 37461,
         'него.': 40,
         'мой': 2927,
         'мальчик': 147,
         'все': 14966,
         'равно': 1284,
         'лучше:d': 5,
         'rt': 37228,
         '@katiacheh:': 1,
         'ну': 10957,
         'ты': 13504,
         'идиотка)': 2,
         'испугалась': 25,
         'за': 10463,
         'тебя!!!': 11,
         '@digger2912:': 1,
         '"кто': 23,
         'в': 55969,
         'углу': 33,
         'сидит': 208,
         'погибает': 3,
   

In [69]:
most_com_set = set([word_counts.most_common(75)[i][0] for i in range(75)])
most_com_set = set([word_counts.most_common(75)[i][0] for i in range(75)])

def f(s):
    s = list(s.split())
    s_l = len(s)
    new_s = []
    for i in range(len(s)):
        if not s[i] in most_com_set:
            new_s.append(s[i])
        
    return ' '.join(new_s)

new_data = data['text'].apply(f)

In [70]:
new_new_data = data.copy()

In [71]:
new_new_data['text'] = new_data

In [72]:
new_new_data

,id,date,mane,text,type,trep,trtw,tfav,stcount,fol,frien,listcount,label
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee школота, поверь, самое :D обществ...",1,0,0,0,7569,62,61,0,1
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки немного похож него. Но мальчик лу...",1,0,0,0,11825,59,31,2,1
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну идиотка) испугалась тебя!!!,1,0,1,0,1273,26,27,0,1
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто углу сидит погибает голод...",1,0,1,0,1549,19,17,0,1
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот значит страшилка :D Но бли...,1,0,0,0,597,16,23,1,1
5,408906761769598976,1386325943,JustinB94262583,любишь нет? Я бля:D http://t.co/brf9eNg1U6,1,0,0,0,40,6,16,0,1
6,408906762436481024,1386325943,twinkleAYO,"RT @SpoonLamer: Ох,900 :D конечно @twinkleAYO ...",1,0,1,0,5169,58,43,2,1
7,408906764114206720,1386325944,pycalyruhog,RT @veregijytaqo: У ухажёр? Нет уши жрёт :D,1,0,2,0,393,112,153,0,1
8,408906764608749568,1386325944,grishintv,Поприветствуем моего нового читателя @Alexey1789,1,0,0,0,5872,1387,1431,12,1
9,408906765841870848,1386325944,alina_612,Теперь частичка Сиднея #Sydney #SydneyOperaHou...,1,0,0,0,263,5,36,0,1


In [17]:
# наименее встречаемые
for word in sorted(word_counts.items(), key=lambda x: x[1])[:20]:
    print(word[0], word[1])

@first_timee 1
профилирующий 1
@katiacheh: 1
@digger2912: 1
http://t.co/gqg6iue2… 1
@irina_dyshkant 1
страшилка 1
блин,посмотрев 1
части,у 1
создастся 1
http://t.co/brf9eng1u6 1
@spoonlamer: 1
ох,900 1
@veregijytaqo: 1
ухажёр? 1
сиднея 1
#sydney 1
#sydneyoperahouse 1
http://t.co/agnto3czei 1
папу!!!!а 1


In [75]:
tfidf = TfidfVectorizer(min_df=2)
f = tfidf.fit_transform(new_new_data["text"])
print(f.shape)

(226834, 102104)


In [76]:
%%time
# делим на обучение и тест
trX, teX, trY, teY, trDF, teDF = train_test_split(
    f, new_new_data["label"], new_new_data, test_size=0.2, stratify=new_new_data["label"], random_state=42
)
clf = LogisticRegression(n_jobs=1, solver='liblinear')
clf.fit(trX, trY)

CPU times: user 1.76 s, sys: 4 ms, total: 1.76 s
Wall time: 1.76 s


In [77]:
print("train accuracy:", accuracy_score(trY, clf.predict(trX)))
print("test accuracy:", accuracy_score(teY, clf.predict(teX)))

train accuracy: 0.8321843640992577
test accuracy: 0.7533890272665154


# Задание

Попробуйте отфильтровать наиболее частотные слова, обучите модель и проверьте качество

## Добавим словосочетания

In [28]:
tfidf = TfidfVectorizer(min_df=5, ngram_range=(1, 2))
f = tfidf.fit_transform(data["text"])
print(f.shape)

(226834, 83403)


In [29]:
%%time
# делим на обучение и тест
trX, teX, trY, teY, trDF, teDF = train_test_split(
    f, data["label"], data, test_size=0.2, stratify=data["label"], random_state=42
)
clf = LogisticRegression(n_jobs=1, solver='liblinear')
clf.fit(trX, trY)

CPU times: user 2 s, sys: 0 ns, total: 2 s
Wall time: 2 s


In [30]:
print("train accuracy:", accuracy_score(trY, clf.predict(trX)))
print("test accuracy:", accuracy_score(teY, clf.predict(teX)))

train accuracy: 0.8365102194889429
test accuracy: 0.7743293583441708


In [31]:
coefs = sorted(zip(list(np.array(tfidf.get_feature_names())[clf.coef_[0] != 0]), clf.coef_[0][clf.coef_[0] != 0]),
               key=lambda x: -x[1])
coefs = pd.DataFrame(coefs, columns=["ngram", "weight"])

In [32]:
coefs.head(30)

,ngram,weight
0,dd,10.735821
1,ddd,8.215684
2,ахах,5.728463
3,ахаха,5.623601
4,приятно,5.474277
5,спасибо,5.365067
6,ахахах,5.127073
7,dddd,5.008089
8,xd,4.791211
9,обожаю,4.309966


In [ ]:
coefs.tail(30)[::-1]

## Задание

Попробуйте использовать 3-граммы, подберите оптимальное min_df и max_df. Обучите модель и замерьте качество (нужно добиться улучшения)

In [38]:
tfidf = TfidfVectorizer(min_df=5, ngram_range=(3, 3))
f = tfidf.fit_transform(data["text"])
print(f.shape)

(226834, 18611)


In [39]:
print("train accuracy:", accuracy_score(trY, clf.predict(trX)))
print("test accuracy:", accuracy_score(teY, clf.predict(teX)))

train accuracy: 0.8365102194889429
test accuracy: 0.7743293583441708


## Дополнительное задание

Придумайте другие признаки на основе текста, обучите логистическую регрессию. Добейтесь наилучшего результата на тестовой выборке

# Word2vec

In [35]:
import gensim

In [34]:
!pip install gensim

    100% |████████████████████████████████| 22.6MB 65kB/s 
    100% |████████████████████████████████| 1.4MB 292kB/s 
    100% |████████████████████████████████| 92kB 88kB/s 
    100% |████████████████████████████████| 133kB 88kB/s 
    100% |████████████████████████████████| 153kB 176kB/s 
    100% |████████████████████████████████| 143kB 168kB/s 
    100% |████████████████████████████████| 61kB 87kB/s 
  Using cached https://files.pythonhosted.org/packages/bd/c9/6fdd990019071a4a32a5e7cb78a1d92c53851ef4f56f62a3486e6a7d8ffb/urllib3-1.23-py2.py3-none-any.whl
    100% |████████████████████████████████| 61kB 86kB/s 
    100% |████████████████████████████████| 4.4MB 116kB/s 
    100% |████████████████████████████████| 552kB 107kB/s 
  Running setup.py bdist_wheel for smart-open ... done
  Stored in directory: /home/maximus/.cache/pip/wheels/b1/9e/7d/bb3d3b55c597e72617140a0638c06382a5f17283881eae163e
  Running setup.py bdist_wheel for bz2file ... done
  Stored in directory: /home/maximus/.c

Загрузим предобученную модель

In [118]:
def f(s):
    s = s.lower()
    new_s = []
    for i in range(len(s)):
        if s[i].isalpha() or s[i].isspace():
            new_s.append(s[i])
    s = ''.join(new_s)
    return list(s.split())

new_new_data= data.copy()
new_new_data['text'] = new_new_data['text'].apply(f)

In [119]:
model = gensim.models.Word2Vec(np.array(new_new_data['text']))

In [104]:
model.wv

(18403018, 226834)

In [86]:
np.array(data['text'])

array(['@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)',
       'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D',
       'RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!', ...,
       'Вот и в школу, в говно это идти уже надо(',
       'RT @_Them__: @LisaBeroud Тауриэль, не грусти :( *обнял*',
       'Такси везет меня на работу. Раздумываю приплатить, чтобы меня втащили на пятый этаж. Лифта то нет :('],
      dtype=object)

Найдем ближайшие слова к словам "король", "отец"

In [120]:
model.most_similar(positive=['мальчик'])

/home/maximus/anaconda3/envs/cs/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('маленький', 0.9040625095367432),
 ('родной', 0.8843885660171509),
 ('мужчина', 0.8755344152450562),
 ('любимый', 0.8716735243797302),
 ('милый', 0.8652970790863037),
 ('одноклассник', 0.8411803245544434),
 ('кот', 0.8380382061004639),
 ('ребенок', 0.8366769552230835),
 ('брат', 0.8361928462982178),
 ('заварочный', 0.8290929198265076)]

In [126]:
model.most_similar(positive=["мама"])

/home/maximus/anaconda3/envs/cs/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('она', 0.8230205774307251),
 ('папа', 0.8105506896972656),
 ('бабушка', 0.7736116647720337),
 ('сестра', 0.7195959091186523),
 ('мамка', 0.6768721342086792),
 ('сказала', 0.6552960872650146),
 ('настя', 0.6498634815216064),
 ('подруга', 0.6379276514053345),
 ('маман', 0.6357685327529907),
 ('позвонила', 0.6351274251937866)]

Можно применять различные линейные преобразования к векторам.

Например, найдем ближайший вектор к вектору "король - мужчина + женщина"

In [121]:
model.most_similar(positive=["король", "женщина"], negative=["мужчина"])

/home/maximus/anaconda3/envs/cs/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('кривые', 0.8502144813537598),
 ('ебаная', 0.8499696254730225),
 ('мода', 0.8498300909996033),
 ('суровая', 0.847414493560791),
 ('молодая', 0.8427489995956421),
 ('знакомая', 0.8416356444358826),
 ('задача', 0.8411151766777039),
 ('комната', 0.8403501510620117),
 ('сплошная', 0.8388898968696594),
 ('трудная', 0.8380223512649536)]

А вот другой пример, поиск антонимов:

"громкий" - "тихий" + "слабый"

In [122]:
model.most_similar(positive=["громкий", "слабый"], negative=["тихий"])

/home/maximus/anaconda3/envs/cs/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('ахахахахахахахахахаха', 0.8532185554504395),
 ('лакшери', 0.8508890271186829),
 ('горит', 0.8477257490158081),
 ('пропадает', 0.8454916477203369),
 ('поёт', 0.8439762592315674),
 ('трезвый', 0.8386509418487549),
 ('хозяин', 0.8363634347915649),
 ('турецкий', 0.8361029028892517),
 ('прервал', 0.8338011503219604),
 ('личность', 0.8326117992401123)]

"громкий" - "тихий" + "высокий"

In [123]:
model.most_similar(positive=["громкий", "высокий"], negative=["тихий"])

/home/maximus/anaconda3/envs/cs/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('секс', 0.8379783034324646),
 ('носит', 0.8377717733383179),
 ('смайлик', 0.8374501466751099),
 ('настоящий', 0.8283652663230896),
 ('умный', 0.8228262662887573),
 ('ахахахахахахахахахаха', 0.8214380741119385),
 ('давид', 0.8193902969360352),
 ('рост', 0.8171775341033936),
 ('намёк', 0.8164290189743042),
 ('сын', 0.8155866861343384)]

# Задание

Придумайте, как, используя линейные преобразования над векторами, на основе страны находить её столицу

In [129]:
model.most_similar(positive=['франция'1, 'москва'], negative=['россия'])

/home/maximus/anaconda3/envs/cs/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('глупая', 0.8695499897003174),
 ('живая', 0.865013599395752),
 ('собачка', 0.8639159798622131),
 ('анька', 0.8595185279846191),
 ('дочка', 0.8580965995788574),
 ('рыжая', 0.8566163778305054),
 ('офигела', 0.8555711507797241),
 ('диана', 0.8545217514038086),
 ('понялчто', 0.8538469076156616),
 ('няшка', 0.8537920713424683)]

Придумайте, как, используя линейные преобразования над векторами, на основе слова определять его множественное число

In [132]:
model.most_similar(positive=['бутылка', 'много'])

/home/maximus/anaconda3/envs/cs/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('дофига', 0.8591175079345703),
 ('полно', 0.8566795587539673),
 ('занимает', 0.8406789302825928),
 ('отсутствие', 0.836215615272522),
 ('глупых', 0.8348427414894104),
 ('умных', 0.8329007029533386),
 ('дохера', 0.8322297930717468),
 ('дохуя', 0.8286299705505371),
 ('набрали', 0.8248424530029297),
 ('необычной', 0.8243659734725952)]

Придумайте, как, используя линейные преобразования над векторами, для прилагательного определять его превосходную форму